<a href="https://colab.research.google.com/github/Ranjani94/Advanced_Deep_Learning/blob/master/Assignment_1_Part_1/Multi_learning_instance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Multiple Learning Instance

It is defined as weakly supervised learning algorithm where training data are split into two bags which is positive and negative bags and one label for each bag. All the instances in the bag is said to be positive if any one instance is positive whereas if only all of the instances are negative the bag is negative. Label for individual instances exist inside bag but are unknown suring the training. 

###Importing required libraries..

In [1]:
import tensorflow as tf
import os
import time
import datetime

import copy
import re
import yaml
import uuid
import warnings
from functools import partial, reduce
from random import shuffle
import random

import numpy as np
import pandas as pd

from sklearn import metrics as mtx
from sklearn import model_selection as ms
from tqdm.autonotebook import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


###Mounting drive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'advanced_DL'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/3wEn1INYsbzrmWe5G9FlrU6yjAJRCp5lFyeHVzVF1Fchswsejp6fuBo
Mounted at /content/gdrive


###Import all the necessary pytorch library

In [3]:
import torch
from torch import nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision.models import resnet
from torchvision.models.resnet import ResNet, BasicBlock
from torchvision.datasets import MNIST

import inspect
import time
from torch import nn, optim
import torch
from torchvision.transforms import Compose, ToTensor, Normalize, Resize
from torch.utils.data import DataLoader

###Loading MNIST Dataset using Pytorch Library

MNIST dataset is a collection of handwritten digits images which has label from 0 to 9. This dataset has 60,000 training set and 10,000 test set. Loading the dataset using pytorch library and defining the batch size as 256 for both training and test. The dataset is resized to 224 X 224 pixel size and normalized by taking mean and standard deviation. Data is split into training and validation data.

In [4]:
def get_data_loaders(training_data_batch_size, valid_data_batch_size):
    mnist = MNIST(download=True, train=True, root=".").train_data.float()
    
    data_transform = Compose([ Resize((224, 224)),ToTensor(), Normalize((mnist.mean()/255,), (mnist.std()/255,))])

    train_data_loader = DataLoader(MNIST(download=True, root=".", transform=data_transform, train=True),
                              batch_size=training_data_batch_size, shuffle=True)
    validation_data_loader = DataLoader(MNIST(download=False, root=".", transform=data_transform, train=False),
                            batch_size=valid_data_batch_size, shuffle=False)
    return train_data_loader, validation_data_loader

In [5]:
valid_data_batch_size = 256
training_data_batch_size = 256

In [6]:
train_data_loader, validation_data_loader = get_data_loaders(training_data_batch_size, valid_data_batch_size)

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


###Defining the ResNet model 

First step is to train the pre trained ResNet model using the labeled dataset that is loaded. Second, will pass the bag labeled dataset to the same model so that we can extra the features. Ant at last we will apply the Multiple Instance Learning model on the dataset.

The Residual bloack has convolutional two dimensional with kernel size 7, stride as (2,2) and paddig as (3,3)

In [7]:
class Residual_Block(ResNet):
    def __init__(self):
        super(Residual_Block, self).__init__(BasicBlock, [2, 2, 2, 2], num_classes=10)
        self.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        
    def forward(self, x):
        return torch.softmax(super(Residual_Block, self).forward(x), dim=-1)

Evalution metrices funtion is used to calculate the difference between true and predicted output values from the model. Scores like precision, recall, F1, accuracy is calculated to evaluate the model performance.

In [8]:
def evaluation_metric(metric_fn, true_y, pred_y):
    if "average" in inspect.getfullargspec(metric_fn).args:
        return metric_fn(true_y, pred_y, average="macro")
    else:
        return metric_fn(true_y, pred_y)
    
def evaluation_scores(p, r, f1, a, batch_size):
    for name, scores in zip(("precision", "recall", "F1", "accuracy"), (p, r, f1, a)):
        print(f"\t{name.rjust(14, ' ')}: {sum(scores)/batch_size:.4f}")

Time for each training time is calculated and the device is run using GPU. The loss function used is Cross Entropy Loss and the Optimizer is adadelta. Losses has been calculated at each training

In [9]:
start_ts = time.time()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
model = Residual_Block().to(device)
epochs = 3
train_data_loader, validation_data_loader = get_data_loaders(training_data_batch_size, valid_data_batch_size)
loss_function = nn.CrossEntropyLoss()         # loss function, cross entropy works well for multi-class problems

# optimizer
optimizer = optim.Adadelta(model.parameters())


losses = []
batches = len(train_data_loader)
val_batches = len(validation_data_loader)

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


###Training the modeland calculating the losses, training loss and validation loss, evaluation metrices and batches

In [11]:
for epoch in range(epochs):
    total_loss = 0
    progress = tqdm(enumerate(train_data_loader), desc="Loss: ", total=batches)
    model.train()
    
    for i, data in progress:
        X, y = data[0].to(device), data[1].to(device)
        model.zero_grad() 
        outputs = model(X)                     
        loss = loss_function(outputs, y)       
        loss.backward()                        
        optimizer.step()                        
        current_loss = loss.item()
        total_loss += current_loss
        progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))
        
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    val_losses = 0
    precision, recall, f1, accuracy = [], [], [], []
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(validation_data_loader):
            X, y = data[0].to(device), data[1].to(device)

            outputs = model(X)                                    
            val_losses += loss_function(outputs, y)

            predicted_classes = torch.max(outputs, 1)[1]          
            
            
            for acc, metric in zip((precision, recall, f1, accuracy), 
                                   (precision_score, recall_score, f1_score, accuracy_score)):
                acc.append(
                    evaluation_metric(metric, y.cpu(), predicted_classes.cpu())
                )
          
    print(f"Epoch {epoch+1}/{epochs}, training loss: {total_loss/batches}, validation loss: {val_losses/val_batches}")
    evaluation_scores(precision, recall, f1, accuracy, val_batches)
    losses.append(total_loss/batches) 
print(f"Training time: {time.time()-start_ts}s")


Epoch 1/3, training loss: 1.6667393009713356, validation loss: 1.5209722518920898
	     precision: 0.9588
	        recall: 0.9518
	            F1: 0.9523
	      accuracy: 0.9528



Epoch 2/3, training loss: 1.4778872672547685, validation loss: 1.5357602834701538
	     precision: 0.9464
	        recall: 0.9323
	            F1: 0.9304
	      accuracy: 0.9326



Epoch 3/3, training loss: 1.4718929290771485, validation loss: 1.5126043558120728
	     precision: 0.9561
	        recall: 0.9542
	            F1: 0.9513
	      accuracy: 0.9554
Training time: 388.18154311180115s


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


###Saving the model

In [12]:
torch.save(model.state_dict(), 'pretrain_minst_model.pt')

###In step 2, loading the pretrained model and extrating the features. First, splitting the remaining dataset into train, validation and test data set. Next we will extra features for training, validation and test data. After extrating feature, we will get bag indices and bag labels. WE will map the bad indices with the features according to the actual indices and finally result in getting the bag features.

In [13]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [14]:
x_train = x_train[:30001]
y_train = y_train[:30001]
x_test = x_test[:9000]
y_test = y_test[:9000]

In [15]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [16]:
x_train /= 255
x_test /= 255

In [17]:
print('x_train shape:', x_train.shape)
print('Number of images in x_train', x_train.shape[0])
print('Number of images in x_test', x_test.shape[0])

x_train shape: (30001, 28, 28)
Number of images in x_train 30001
Number of images in x_test 9000


###Creating the tuple set which has index and label for both training and test set

In [18]:
instance_index_label = [(i, y_train[i]) for i in range(x_train.shape[0])]
instance_index_label_test = [(i, y_test[i]) for i in range(x_test.shape[0])]

In [19]:
# find the index if label is 1
find_index = [instance_index_label[i][0] for i in range(len(instance_index_label)) if instance_index_label[i][1]==1]
# find the index if label is 1
find_index_test = [instance_index_label_test[i][0] for i in range(len(instance_index_label_test))
                   if instance_index_label_test[i][1]==1]

In [20]:
print('index:', instance_index_label[0][0]) #index
print('label:', instance_index_label[0][1]) #label

index: 0
label: 5


###Loading the pretrained resnet model to extra the features

In [21]:

import torch
from torchvision.models.resnet import ResNet, BasicBlock
class Residual_Block(ResNet):
    def __init__(self):
        super(Residual_Block, self).__init__(BasicBlock, [2, 2, 2, 2], num_classes=10)
        self.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        
    def forward(self, x):
        return torch.softmax(super(Residual_Block, self).forward(x), dim=-1)

In [22]:
model = Residual_Block()
model.load_state_dict(torch.load('pretrain_minst_model.pt'))
body = nn.Sequential(*list(model.children()))
model = body[:9]
model.eval()

Sequential(
  (0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

###Getting the features for training, validation and test set. Using the Adam optimizer and passing the batch size 1 for both training and validation data


In [23]:
train_batch_size = 1
val_batch_size = 1
train_data_loader, validation_data_loader = get_data_loaders(train_batch_size, 1)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adadelta(model.parameters())

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [24]:
losses = []
batches = len(train_data_loader)
val_batches = len(validation_data_loader)

In [25]:
batches

60000

In [26]:
val_batches

10000

###Getting feature set for the training data

In [27]:
meta_table = dict()
feature_result = []

# progress bar
progress = tqdm(enumerate(train_data_loader), desc="Loss: ", total=batches)

model.eval() #model

for i, data in progress:
    if i==30001:
        break
    X, y = data[0], data[1]
    
    model.zero_grad()
    outputs = model(X)
    feature_result.append(outputs.reshape(-1).tolist())
    meta_table[i] = outputs.reshape(-1).tolist()
    
feature_array = np.array(feature_result)
np.save('feature_array_full',feature_array)

###Getting features for the test set

In [28]:
meta_t_table = dict()
feature_t_result = []

# progress bar
progress = tqdm(enumerate(validation_data_loader), desc="Loss: ", total=batches)

model.eval()

for i, data in progress:
    if i==9000:
        break
    X, y = data[0], data[1]
    # training step for single batch
    model.zero_grad()
    outputs_t = model(X)
    feature_t_result.append(outputs_t.reshape(-1).tolist())
    meta_t_table[i] = outputs_t.reshape(-1).tolist()

feature_test_array = np.array(feature_t_result)
# save 
np.save('feature_test_array_full',feature_test_array )

In [29]:
feature_array = np.load('feature_array_full.npy', allow_pickle=True)

In [30]:
feature_array.shape

(30001, 512)

###Loading the saved feature array for test data

In [31]:
feature_test_array = np.load('feature_test_array_full.npy', allow_pickle=True)

In [32]:
feature_test_array.shape

(9000, 512)

###Creating bags and bag label for training data

In [33]:
from typing import List, Dict, Tuple
def bag_generation(ins_train_label: List[Tuple]) -> List[Dict]:
    """
    bags: {key1: [ind1, ind2, ind3],
           key2: [ind1, ind2, ind3, ind4, ind5],
           ... }
    bag_lbls:
        {key1: 0,
         key2: 1,
         ... }
    """
    bag_size = np.random.randint(3,7,size=len(ins_train_label)//5)
    data_cp = copy.copy(ins_train_label)
    np.random.shuffle(data_cp)
    bags = {}
    bags_per_instance_labels = {}
    bags_labels = {}
    for bag_ind, size in enumerate(bag_size):
        bags[bag_ind] = []
        bags_per_instance_labels[bag_ind] = []
        try:
            for _ in range(size):
                inst_ind, lbl = data_cp.pop()
                bags[bag_ind].append(inst_ind)
                # simplfy, just use a temporary variable instead of bags_per_instance_labels
                bags_per_instance_labels[bag_ind].append(lbl)
            bags_labels[bag_ind] = bag_label_from_instance_labels(bags_per_instance_labels[bag_ind])
        except:
            break
    return bags, bags_labels

def bag_label_from_instance_labels(instance_labels):
    return int(any(((x==1) for x in instance_labels)))

In [34]:
bag_indices, bag_labels = bag_generation(instance_index_label)
bag_features = {kk: torch.Tensor(feature_array[inds]) for kk, inds in bag_indices.items()}

###Saving bag indices, labels and features using pickle library

In [35]:
import pickle
pickle.dump(bag_indices, open( "bag_indices", "wb" ) )
pickle.dump(bag_labels, open( "bag_labels", "wb" ) )
pickle.dump(bag_features, open( "bag_features", "wb" ) )

###Loading the saved bag features..


In [36]:
bag_indices = pickle.load( open( "bag_indices", "rb" ) )
bag_labels = pickle.load( open( "bag_labels", "rb" ) )
bag_features = pickle.load( open( "bag_features", "rb" ) )

###Generating bag for the testing data

In [37]:
bag_test_indices, bag_test_labels = bag_generation(instance_index_label_test)

In [38]:
bag_test_features = {kk: torch.Tensor(feature_test_array[inds]) for kk, inds in bag_test_indices.items()}

In [39]:
pickle.dump(bag_test_indices, open( "bag_test_indices", "wb" ) )
pickle.dump(bag_test_labels, open( "bag_test_labels", "wb" ) )
pickle.dump(bag_test_features, open( "bag_test_features", "wb" ) )

In [40]:
bag_test_indices = pickle.load( open( "bag_test_indices", "rb" ) )
bag_test_labels = pickle.load( open( "bag_test_labels", "rb" ) )
bag_test_features = pickle.load( open( "bag_test_features", "rb" ) )

#Multiple Instance Learning

In [41]:
class Transform_data(Dataset):
 
    def __init__(self, data, transform=None):
        self.transform = transform
        self.data = data
        
    def __getitem__(self, index):
        tensor = self.data[index][0]
        if self.transform is not None:
            tensor = self.transform(tensor)
        return (tensor, self.data[index][1])

    def __len__(self):
        return len(self.data)

In [42]:
training_data = [(bag_features[i],bag_labels[i]) for i in range(len(bag_features))]

In [43]:
bag_features[0]

tensor([[1.9886, 0.9590, 0.1920,  ..., 0.0586, 0.1633, 0.4931],
        [1.3405, 1.2323, 1.2333,  ..., 1.5910, 1.8635, 0.7699],
        [1.4816, 1.8155, 0.1413,  ..., 0.0910, 0.1902, 0.2798]])

Since our bag has different sizes, we need to pad each tensor to have the same shape (max: 7). We will look through each one instance and look at the shape of the tensor, and then we will pad 7-n to the existing tensor where n is the number of instances in the bag. The function will return a padded data set.

In [44]:
def pad_tensor(data:list, max_number_instance) -> list:
    new_data = []
    for bag_index in range(len(data)):
        tensor_size = len(data[bag_index][0])
        pad_size = max_number_instance - tensor_size
        p2d = (0,0, 0, pad_size)
        padded = nn.functional.pad(data[bag_index][0], p2d, 'constant', 0)
        new_data.append((padded, data[bag_index][1]))
    return new_data

In [45]:
max_number_instance = 7
padded_train = pad_tensor(training_data, max_number_instance)

In [46]:
test_data = [(bag_test_features[i],bag_test_labels[i]) for i in range(len(bag_test_features))]
padded_test = pad_tensor(test_data, max_number_instance)

In [47]:
def get_data_loaders(training_data, test_data, train_batch_size, val_batch_size):
    train_loader = DataLoader(training_data, batch_size=train_batch_size, shuffle=True)
    valid_loader = DataLoader(test_data, batch_size=val_batch_size, shuffle=False)
    return train_loader, valid_loader

In [48]:
train_loader,valid_loader = get_data_loaders(padded_train, padded_test, 1, 1)

In [49]:

train_batch_size = 1
val_batch_size = 1

###Defining the model and using attention softmax as aggregation functions

In [50]:
class SoftMaxMeanSimple(torch.nn.Module):
    def __init__(self, n, n_inst, dim=0):
        super(SoftMaxMeanSimple, self).__init__()
        self.dim = dim
        self.gate = torch.nn.Softmax(dim=self.dim)      
        self.mdl_instance_transform = nn.Sequential(
                            nn.Linear(n, n_inst),
                            nn.LeakyReLU(),
                            nn.Linear(n_inst, n),
                            nn.LeakyReLU(),
                            )
    def forward(self, x):
        z = self.mdl_instance_transform(x)
        if self.dim==0:
            z = z.view((z.shape[0],1)).sum(1)
        elif self.dim==1:
            z = z.view((1, z.shape[1])).sum(0)
        gate_ = self.gate(z)
        res = torch.sum(x* gate_, self.dim)
        return res, gate_

    
class AttentionSoftMax(torch.nn.Module):
    def __init__(self, in_features = 3, out_features = None):
        super(AttentionSoftMax, self).__init__()
        self.otherdim = ''
        if out_features is None:
            out_features = in_features
        self.layer_linear_tr = nn.Linear(in_features, out_features)
        self.activation = nn.LeakyReLU()
        self.layer_linear_query = nn.Linear(out_features, 1)
        
    def forward(self, x):
        keys = self.layer_linear_tr(x)
        keys = self.activation(keys)
        attention_map_raw = self.layer_linear_query(keys)[...,0]
        attention_map = nn.Softmax(dim=-1)(attention_map_raw)
        result = torch.einsum(f'{self.otherdim}i,{self.otherdim}ij->{self.otherdim}j', attention_map, x)
        return result, attention_map

###The MIL algorithm ahs three steps. First we have to transform the instance to get the embeddings and pass them through a aggregation function.At last transform them into a bag probability. Using linear layer and LeakyReLu activation function 

# updated

In [51]:
class linear(torch.nn.Module):

    def __init__(self, n=7*512, n_out=1, dropout=0.2):
        super(linear, self).__init__()
        self.linear1 = torch.nn.Linear(n, n_out)
        
    def forward(self, x):
        z = self.linear1(x)
        y_pred = torch.sigmoid(z)
        return y_pred

In [52]:
class NN(torch.nn.Module):

    def __init__(self, n=7*512, n_mid = 7168, n_out=1, dropout=0.2):
        super(NN, self).__init__()
        self.linear1 = torch.nn.Linear(n, n_mid)
        self.linear2 = torch.nn.Linear(n_mid, n_out)
        self.dropout = torch.nn.Dropout(dropout)
        self.non_linearity = torch.nn.LeakyReLU()
        
    def forward(self, x):
        z = self.linear1(x)
        z = self.non_linearity(z)
        z = self.dropout(z)
        z = self.linear2(z)
        y_pred = torch.sigmoid(z)
        return y_pred

In [53]:
class NoisyAnd(torch.nn.Module):
    def __init__(self, a=10, dims=[0]):
        super(NoisyAnd, self).__init__()
        self.a = a
        self.b = torch.nn.Parameter(torch.tensor(0.01))
        self.dims =dims
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        mean = torch.mean(x, self.dims, False)
        res = (self.sigmoid(self.a * (mean - self.b)) - self.sigmoid(-self.a * self.b)) / (
              self.sigmoid(self.a * (1 - self.b)) - self.sigmoid(-self.a * self.b))
        return res
    

class MIL_NN(torch.nn.Module):

    def __init__(self, n=7*512,  n_mid=7168, n_out=1, 
                 n_inst=None, dropout=0.1,
                 noisy_a=4,
                 agg = NoisyAnd(a=4, dims=[0]),
                ):
        super(MIL_NN, self).__init__()
        if agg is None:
            print("agg is called ")
            agg = NoisyAnd(a=noisy_a, dims=[0])
        if n_inst is None:
            self.mdl_instance = agg
            n_inst = n
        else:
            self.mdl_instance = nn.Sequential(
                            nn.Linear(n, n_inst),
                            nn.LeakyReLU(),
                            agg,
                            )
        if n_mid == 0:
            self.mdl_bag = LogisticRegression(n_inst, n_out)
        else:
            self.mdl_bag = NN(n_inst, n_mid, n_out, dropout=dropout)
        
    def forward(self, bag_feature):

        y_pred = self.mdl_bag(bag_feature)
        return y_pred

In [54]:
def calculate_metric(metric_fn, true_y, pred_y):
    # multi class problems need to have averaging method
    if "average" in inspect.getfullargspec(metric_fn).args:
        return metric_fn(true_y, pred_y, average="macro")
    else:
        return metric_fn(true_y, pred_y)
    
def print_scores(p, r, f1, a, batch_size):
    # just an utility printing function
    for name, scores in zip(("precision", "recall", "F1", "accuracy"), (p, r, f1, a)):
        print(f"\t{name.rjust(14, ' ')}: {sum(scores)/batch_size:.4f}")

###Trainig and testing the MNIST dataset using Multiple Learning Instance(MIL)

In [ ]:
import numpy as np
start_ts = time.time()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

lr0 = 1e-4

# model:
model = MIL_NN().to(device)

# params you need to specify:
epochs = 10
train_loader, val_loader = get_data_loaders(padded_train, padded_test, 1, 1)
loss_function = torch.nn.BCELoss(reduction='mean')      # loss function: BCELoss works well for binary class problems


#optimizer
optimizer = optim.SGD(model.parameters(), lr=lr0, momentum=0.9)

losses = []
batches = len(train_loader)
val_batches = len(val_loader)

# loop for every epoch (training + evaluation)
for epoch in range(epochs):
    total_loss = 0

    # progress bar
    progress = tqdm(enumerate(train_loader), desc="Loss: ", total=batches)

    # ----------------- TRAINING  -------------------- 
    # set model to training
    model.train()
    for i, data in progress:
        X, y = data[0].to(device), data[1].to(device)
        #print(X.shape)
        X = X.reshape([1,7*512])
        y = y.type(torch.cuda.FloatTensor)
        # training step for single batch
        model.zero_grad() # to make sure that all the grads are 0 
        """
        model.zero_grad() and optimizer.zero_grad() are the same 
        IF all your model parameters are in that optimizer. 
        I found it is safer to call model.zero_grad() to make sure all grads are zero, 
        e.g. if you have two or more optimizers for one model.

        """
        outputs = model(X)                             # forward
        loss = loss_function(outputs, y)               # get loss
        loss.backward()                                # accumulates the gradient (by addition) for each parameter.
        optimizer.step()                               # performs a parameter update based on the current gradient 

        # getting training quality data
        current_loss = loss.item()
        total_loss += current_loss

        # updating progress bar
        progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))
        
    # releasing unceseccary memory in GPU
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    # ----------------- VALIDATION  ----------------- 
    val_losses = 0
    precision, recall, f1, accuracy = [], [], [], []
    
    # set model to evaluating (testing)
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            X, y = data[0].to(device), data[1].to(device)
            X = X.reshape([1,7*512])
            y = y.type(torch.cuda.FloatTensor)
            outputs = model(X)                         # this get's the prediction from the network
            prediced_classes =outputs.detach().round()
            val_losses += loss_function(outputs, y)
            
            # calculate P/R/F1/A metrics for batch
            for acc, metric in zip((precision, recall, f1, accuracy), 
                                   (precision_score, recall_score, f1_score, accuracy_score)):
                acc.append(
                    calculate_metric(metric, y.cpu(), prediced_classes.cpu())
                )
          
    print(f"Epoch {epoch+1}/{epochs}, training loss: {total_loss/batches}, validation loss: {val_losses/val_batches}")
    print_scores(precision, recall, f1, accuracy, val_batches)
    losses.append(total_loss/batches)                  # for plotting learning curve
print(f"Training time: {time.time()-start_ts}s")

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10, training loss: 0.7055807490932444, validation loss: 0.6944136023521423
	     precision: 0.5467
	        recall: 0.5467
	            F1: 0.5467
	      accuracy: 0.5467



Epoch 2/10, training loss: 0.673375070070227, validation loss: 0.6736745238304138
	     precision: 0.5844
	        recall: 0.5844
	            F1: 0.5844
	      accuracy: 0.5844



Epoch 3/10, training loss: 0.661534097964565, validation loss: 0.6735872030258179
	     precision: 0.5739
	        recall: 0.5739
	            F1: 0.5739
	      accuracy: 0.5739


...


Epoch 5/10, training loss: 0.6445469487719238, validation loss: 0.6749505996704102
	     precision: 0.5917
	        recall: 0.5917
	            F1: 0.5917
	      accuracy: 0.5917



Epoch 6/10, training loss: 0.6340394329888125, validation loss: 0.6918259859085083
	     precision: 0.5611
	        recall: 0.5611
	            F1: 0.5611
	      accuracy: 0.5611



Epoch 7/10, training loss: 0.6244901318972309, validation loss: 0.7116829752922058
	     precision: 0.5667
	        recall: 0.5667
	            F1: 0.5667
	      accuracy: 0.5667



Epoch 8/10, training loss: 0.6124915553716321, validation loss: 0.6989603042602539
	     precision: 0.5817
	        recall: 0.5817
	            F1: 0.5817
	      accuracy: 0.5817



Epoch 9/10, training loss: 0.6054384255545835, validation loss: 0.710227906703949
	     precision: 0.5561
	        recall: 0.5561
	            F1: 0.5561
	      accuracy: 0.5561
